In [5]:
import pandas as pd
import tensorflow as tf
import torch
import numpy as np
from tqdm import tqdm
from simpletransformers.t5 import T5Model
from datasets import load_dataset, DownloadMode, load_metric
from transformers import T5Tokenizer

translate_template_de_en = lambda x:  "Translate '%s' from english to german"%x

# train_df = load_dataset("wmt16", "de-en", split="train[:100000]", num_proc=8)
df = pd.read_csv('task.csv')

# Create data frame
# df = pd.DataFrame({
#     "input_text":[translate_template_de_en(train_df[i]['translation']['en']) for i in tqdm(range(len(train_df))) ],
#     "target_text":[train_df[i]['translation']['en'] for i in tqdm(range(len(train_df))) ],
#     "prefix":np.array(["mx_model_selector"]*len(train_df))
# }).applymap(str)

df.head()

,input_text,origin_txt,w_task_set,output_txt
0,Translate 'Wiederaufnahme der Sitzungsperiode'...,Wiederaufnahme der Sitzungsperiode,translate_de_en,Resumption of the session
1,"Translate 'Ich erkläre die am Freitag, dem 17....","Ich erkläre die am Freitag, dem 17. Dezember u...",translate_de_en,I declare resumed the session of the European ...
2,"Translate 'Wie Sie feststellen konnten, ist de...","Wie Sie feststellen konnten, ist der gefürchte...",translate_de_en,"Although, as you will have seen, the dreaded '..."
3,Translate 'Im Parlament besteht der Wunsch nac...,Im Parlament besteht der Wunsch nach einer Aus...,translate_de_en,You have requested a debate on this subject in...
4,Translate 'Heute möchte ich Sie bitten - das i...,Heute möchte ich Sie bitten - das ist auch der...,translate_de_en,"In the meantime, I should like to observe a mi..."


In [13]:
df.rename({'w_task_set':'prefix','origin_txt':'target_text'}, inplace=True, axis=1)
df=df.sample(frac=1)
df.head()

,input_text,target_text,prefix,output_txt
35514,Which English Lord led the Charge of the Light...,Which English Lord led the Charge of the Light...,TriviaQA,"{'aliases': ['Earl of cardigan', 'Earls of Car..."
29757,Who wrote the song Momma Told Me Not To Come?,Who wrote the song Momma Told Me Not To Come?,TriviaQA,"{'aliases': ['Randy Numan', 'Randy newman', 'R..."
33686,Who is most commonly attributed as finding Ame...,Who is most commonly attributed as finding Ame...,TriviaQA,"{'aliases': ['Christoffa Corombo', 'Christophe..."
11171,"Translate 'I often have the impression, unlike...","I often have the impression, unlike the countr...",translate_en_de,"Ich habe manchmal den Eindruck, im Gegensatz z..."
24898,True or False: is al(oh)3 a strong base,is al(oh)3 a strong base,BoolQ,False


In [14]:
df['input_text'][0]

"Translate 'Wiederaufnahme der Sitzungsperiode' from german to english"

In [15]:
from transformers import pipeline
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 64,
    "num_train_epochs": 4,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": False,
    "fp16": True,
    "wandb_project": "T5-BART",
}

model = T5Model("t5", "t5-small", args=model_args)
torch.cuda.empty_cache()
model.train_model(df)

/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


  0%|          | 0/47978 [00:00<?, ?it/s]

/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3712: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Training loss,██▂▁▂▁▁▁▁▁
global_step,▁▂▃▃▄▅▆▆▇█
lr,▁▂▃▄▅▆▇███
Training loss,0.00711
global_step,500
lr,0.001


...

Running Epoch 1 of 4:   0%|          | 0/750 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/750 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/750 [00:00<?, ?it/s]

(3000, 0.01995186731851815)

--- Logging error ---
Traceback (most recent call last):
  File "/home/agupt135/anaconda3/lib/python3.9/logging/__init__.py", line 1087, in emit
    self.flush()
  File "/home/agupt135/anaconda3/lib/python3.9/logging/__init__.py", line 1067, in flush
    self.stream.flush()
OSError: [Errno 12] Cannot allocate memory
Call stack:
  File "/home/agupt135/anaconda3/lib/python3.9/threading.py", line 937, in _bootstrap
    self._bootstrap_inner()
  File "/home/agupt135/anaconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/agupt135/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/home/agupt135/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/home/agupt135/anaconda3/lib/python3.9/site-packages/wandb/sdk/internal/internal.py", line 280, in _process
    self._hm.handle(record)
  File "/home/agupt135/anaconda

In [21]:
print(model.predict(["Translate 'Wiederaufnahme der Sitzungsperiode' from german to english"]))

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Wiederaufnahme der Sitzungsperiode']


In [19]:
# Save model
import os

output_dir="./T5toBART/"
path_model = os.path.join(output_dir, "model")
path_args = os.path.join(output_dir, "model_files")
model.save_model(path_model)
model.save_model_args(path_args)